In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import io

# --- PARTIE 0 : CHARGEMENT DU FICHIER BRUT ---
print("### BLOC 0 : CHARGEMENT DU FICHIER BRUT ###")

### BLOC 0 : CHARGEMENT DU FICHIER BRUT ###


In [ ]:
# 1. TÉLÉCHARGEMENT DU FICHIER
# Vous devez sélectionner 'bike_share_dirty_60.csv' lors de l'exécution
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# 2. CHARGEMENT DANS UN DATAFRAME
df = pd.read_csv(io.BytesIO(uploaded[file_name]))

Saving bike_share_dirty_60.csv to bike_share_dirty_60 (3).csv


In [ ]:
print(f"Fichier chargé. Nombre initial de lignes: {len(df)}")
print(df[['trip_id', 'duration_seconds', 'start_station_id', 'user_type']].head(35))
print("-" * 70)

Fichier chargé. Nombre initial de lignes: 60
    trip_id  duration_seconds start_station_id    user_type
0         1             282.0               S3       Abonné
1         2            2413.0               S1       Abonné
2         3            1132.0               S5       Abonné
3         4            2412.0               S4       Abonné
4         5             206.0               S2       Abonné
5         6            -120.0               S1       Abonné
6         7            1588.0               S5  Occasionnel
7         8             691.0               S4       Abonné
8         9            1661.0               S5  Occasionnel
9        10            1113.0               S3  Occasionnel
10       11               NaN               S4       Abonné
11       12            1273.0               S1       Abonné
12       13            2073.0               S4  Occasionnel
13       14            2461.0               S2  Occasionnel
14       15             311.0               S3       Ab

In [ ]:
# --- PARTIE 1 : GESTION DES ANOMALIES NUMÉRIQUES (DURÉE ET OUTLIERS) ---
print("### BLOC 1 : CORRECTION DES ANOMALIES DANS 'duration_seconds' ###")

# 1.1 Nettoyage des valeurs négatives (Erreur de capteur)
# Remplacer les durées <= 0 par NaN pour les exclure du calcul
df['duration_seconds'] = df['duration_seconds'].apply(lambda x: np.nan if x <= 0 else x)
print("-> Durées négatives remplacées par NaN.")

### BLOC 1 : CORRECTION DES ANOMALIES DANS 'duration_seconds' ###
-> Durées négatives remplacées par NaN.


In [ ]:
# 1.2 Gestion des Valeurs Aberrantes (Outliers)
# L'outlier était 10000s. Remplacer par la limite supérieure acceptable (2700s)
MAX_DURATION_SECONDS = 2700
df['duration_seconds'] = df['duration_seconds'].apply(lambda x: MAX_DURATION_SECONDS if x > MAX_DURATION_SECONDS else x)
print(f"-> Outliers supérieurs à {MAX_DURATION_SECONDS}s plafonnés.")

-> Outliers supérieurs à 2700s plafonnés.


In [ ]:
# 1.3 Imputation des valeurs manquantes dans la durée
# Remplacer les NaN restants par la moyenne
mean_duration = df['duration_seconds'].mean()
df['duration_seconds'] = df['duration_seconds'].fillna(mean_duration)
print(f"-> NaN de la durée imputés par la moyenne ({mean_duration:.2f}s).")

-> NaN de la durée imputés par la moyenne (1441.38s).


In [ ]:
print("\nAperçu après correction des durées (les anomalies numériques sont traitées) :")
print(df[['trip_id', 'duration_seconds']].loc[[5, 10, 25]]) # Affiche les lignes où les problèmes ont été corrigés (trip_id 5, 10, 25)
print("-" * 70)


Aperçu après correction des durées (les anomalies numériques sont traitées) :
    trip_id  duration_seconds
5         6        1441.37931
10       11        1441.37931
25       26        2700.00000
----------------------------------------------------------------------


In [ ]:
# --- PARTIE 2 : GESTION DES VALEURS MANQUANTES ET TYPAGE ---
print("### BLOC 2 : ASSURANCE QUALITÉ DES DIMENSIONS (Stations & User Type) ###")

# 2.1 Suppression des trajets non identifiables (Station ID manquante ou erronée)
# On supprime les lignes où les IDs de station sont manquants.
rows_before_drop = len(df)
df.dropna(subset=['start_station_id', 'end_station_id'], inplace=True)
rows_after_drop = len(df)
print(f"-> {rows_before_drop - rows_after_drop} ligne(s) supprimée(s) à cause d'ID de station manquant ou erroné.")
df['end_station_id'].replace('MISSING', 'INCONNU', inplace=True)
df['end_station_name'].fillna('Destination Inconnue', inplace=True)
# On met des coordonnées neutres pour éviter que la carte plante, si on souhaite quand même afficher le départ
df['end_lat'].fillna(0, inplace=True)
df['end_lon'].fillna(0, inplace=True)

### BLOC 2 : ASSURANCE QUALITÉ DES DIMENSIONS (Stations & User Type) ###
-> 1 ligne(s) supprimée(s) à cause d'ID de station manquant ou erroné.


/tmp/ipython-input-57958956.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['end_station_id'].replace('MISSING', 'INCONNU', inplace=True)
/tmp/ipython-input-57958956.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [ ]:
#2.2 Correction du problème de type (Typage) dans 'user_type'
# La valeur '999' numérique est remplacée par 'Inconnu'.
df['user_type'] = df['user_type'].astype(str).str.replace('999', 'Inconnu')
print("-> Correction du typage de 'user_type' (999 remplacé par 'Inconnu').")

-> Correction du typage de 'user_type' (999 remplacé par 'Inconnu').


In [ ]:
print("\nAperçu après gestion des manquants/typage (Vérifiez qu'il n'y a plus de NaN dans les colonnes ID) :")
print(df[['trip_id', 'start_station_id', 'end_station_id', 'user_type']].loc[[1, 30]]) # Affiche une ligne propre et la ligne avec 'Inconnu'
print(f"Nombre de lignes finales dans le DataFrame : {len(df)}")
print("-" * 70)


Aperçu après gestion des manquants/typage (Vérifiez qu'il n'y a plus de NaN dans les colonnes ID) :
    trip_id start_station_id end_station_id user_type
1         2               S1             S5    Abonné
30       31               S3             S5   Inconnu
Nombre de lignes finales dans le DataFrame : 59
----------------------------------------------------------------------


In [ ]:
# --- PARTIE 3 : CRÉATION DE NOUVELLES DIMENSIONS (TRANSFORMATION) ---
print("### BLOC 3 : TRANSFORMATION ET FEATURE ENGINEERING POUR POWER BI ###")

# 3.1 Conversion et Extraction Temporelle
df['start_time'] = pd.to_datetime(df['start_time'])
df['DayOfWeek'] = df['start_time'].dt.day_name()
df['Hour'] = df['start_time'].dt.hour
print("-> Extraction des dimensions temporelles (Heure, Jour de la semaine) réussie.")
# 3.2 Conversion d'Unité (Secondes -> Minutes)
df['duration_minutes'] = df['duration_seconds'] / 60
print("-> Création de 'duration_minutes' (unité plus lisible) réussie.")

print("\nAperçu des nouvelles colonnes de transformation :")
print(df[['start_time', 'duration_seconds', 'duration_minutes', 'DayOfWeek', 'Hour']].head())
print("-" * 70)

### BLOC 3 : TRANSFORMATION ET FEATURE ENGINEERING POUR POWER BI ###
-> Extraction des dimensions temporelles (Heure, Jour de la semaine) réussie.
-> Création de 'duration_minutes' (unité plus lisible) réussie.

Aperçu des nouvelles colonnes de transformation :
           start_time  duration_seconds  duration_minutes DayOfWeek  Hour
0 2024-07-15 16:54:07             282.0          4.700000    Monday    16
1 2024-07-15 07:44:43            2413.0         40.216667    Monday     7
2 2024-07-15 07:35:13            1132.0         18.866667    Monday     7
3 2024-07-15 17:05:44            2412.0         40.200000    Monday    17
4 2024-07-15 10:44:51             206.0          3.433333    Monday    10
----------------------------------------------------------------------


In [ ]:
# Affichage final

print("--- Data propre pour Power BI ---")

print(df.head())

print(f"Nombre de lignes finales : {len(df)}")



--- Data propre pour Power BI ---
   trip_id          start_time  duration_seconds start_station_id  \
0        1 2024-07-15 16:54:07             282.0               S3   
1        2 2024-07-15 07:44:43            2413.0               S1   
2        3 2024-07-15 07:35:13            1132.0               S5   
3        4 2024-07-15 17:05:44            2412.0               S4   
4        5 2024-07-15 10:44:51             206.0               S2   

  end_station_id user_type start_station_name  start_lat  start_lon  \
0             S2    Abonné         Vieux Lyon    45.7450      4.805   
1             S5    Abonné       Centre Ville    45.7578      4.832   
2             S1    Abonné     Quais du Rhône    45.7350      4.820   
3             S2    Abonné         Université    45.7700      4.840   
4             S5    Abonné     Gare Part-Dieu    45.7600      4.850   

  end_station_name  end_lat  end_lon DayOfWeek  Hour  duration_minutes  
0   Gare Part-Dieu  45.7600    4.850    Monday    1

In [ ]:
# --- PARTIE 4 : EXPORTATION DU JEU DE DONNÉES PROPRE ---
print("### BLOC 4 : EXPORTATION FINALE ###")

# Re-exportation (pour Power BI)
final_csv_name = 'bike_share_final_clean.csv'
df.to_csv(final_csv_name, index=False)
files.download(final_csv_name)

print(f"\n✅ Le fichier FINAL et PROPRE ('{final_csv_name}') est prêt à être analysé dans Power BI.")

### BLOC 4 : EXPORTATION FINALE ###


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Le fichier FINAL et PROPRE ('bike_share_final_clean.csv') est prêt à être analysé dans Power BI.
